Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [42]:
import pandas as pd
import numpy as np

data = pd.read_csv('data-logistic.csv', names=[0, 1, 2])

xs = data[[1, 2]].to_numpy()
y = data[[0]].to_numpy()

Реализуйте градиентный спуск для обычной и L2-регуляризованной
(с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения
используйте вектор (0, 0).

In [43]:
def grad_f(w, y, xs, c, k):
    w1 = w[0]
    w2 = w[1]
    s1 = 0
    s2 = 0
    n = xs.shape[0]
    for i in range(n):
        x = xs[i]
        s1 += y[i] * x[0] * (1 - 1 / (1 + np.exp(-y[i]*(w1 * x[0] + w2 * x[1]))))
        s2 += y[i] * x[1] * (1 - 1 / (1 + np.exp(-y[i]*(w1 * x[0] + w2 * x[1]))))
    return np.array([(w1 + k / n * s1 - k * c * w1)[0], (w2 + k / n * s2 - k * c * w2)[0]])
        
  

In [44]:
def grad_decent(c, k=0.1, w=np.array([0, 0])):
  pred_w = w
  next_w = grad_f(w, y, xs, c, k)
  iters = 0
  while (np.abs(pred_w - next_w)[0] > 10 ** -5) and (np.abs(pred_w - next_w)[1] > 10 ** -5) and iters < 10000:
    iters += 1
    pred_w = next_w
    next_w = grad_f(pred_w, y, xs, c, k)

  return next_w

Запустите градиентный спуск и доведите до сходимости (евклидово
расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число
итераций десятью тысячами.

In [45]:
w_reg = grad_decent(10)

print(w_reg)

[0.02855875 0.02478014]


In [46]:
def a(x, w):
  return 1 / (1 + np.exp(-np.dot(x, w)))

Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на
задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом.
Для этого воспользуйтесь сигмоидной функцией: a(x) = 1/(1 +
exp(−w1x1 − w2x2)

In [47]:
from sklearn.metrics import roc_auc_score


w_withoutreg = grad_decent(0)

y_no_reg = np.expand_dims(a(xs, w_withoutreg), 1)
y_reg = np.expand_dims(a(xs, w_reg), 1)

print(f'Без регуляризации {roc_auc_score(y, np.round(y_no_reg))}')
print(f'С регуляризацией {roc_auc_score(y, np.round(y_reg))}')



Без регуляризации 0.7311904761904762
С регуляризацией 0.7211904761904762


In [48]:
def test_params(k=0.1, c=10, w0=np.array([0, 0])):
  w = grad_decent(c, k, w0)
  y_pred = a(xs, w)
  return roc_auc_score(y, np.round(y_pred))


Попробуйте поменять длину шага. Будет ли сходиться алгоритм,
если делать более длинные шаги? Как меняется число итераций
при уменьшении длины шага? Попробуйте менять начальное приближение. Влияет ли оно на чтонибудь?

In [50]:
print(test_params(0.1, 10, np.array([50, 50])))
print(test_params(0.01, 10, np.array([50, 50])))
print(test_params(0.01, 10, np.array([10, 10])))




C:\Users\пк\AppData\Local\Temp\ipykernel_8184\3702263714.py:9: RuntimeWarning: overflow encountered in exp
  s1 += y[i] * x[0] * (1 - 1 / (1 + np.exp(-y[i]*(w1 * x[0] + w2 * x[1]))))
C:\Users\пк\AppData\Local\Temp\ipykernel_8184\3702263714.py:10: RuntimeWarning: overflow encountered in exp
  s2 += y[i] * x[1] * (1 - 1 / (1 + np.exp(-y[i]*(w1 * x[0] + w2 * x[1]))))
C:\Users\пк\AppData\Local\Temp\ipykernel_8184\3702263714.py:9: RuntimeWarning: overflow encountered in exp
  s1 += y[i] * x[0] * (1 - 1 / (1 + np.exp(-y[i]*(w1 * x[0] + w2 * x[1]))))
C:\Users\пк\AppData\Local\Temp\ipykernel_8184\3702263714.py:10: RuntimeWarning: overflow encountered in exp
  s2 += y[i] * x[1] * (1 - 1 / (1 + np.exp(-y[i]*(w1 * x[0] + w2 * x[1]))))


0.7211904761904762
0.7211904761904762
0.7211904761904762
